In [1]:
import torch
from torchvision.models import convnext_base, ConvNeXt_Base_Weights
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.datasets import CIFAR10
from tqdm import tqdm
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader, SequentialSampler

import matplotlib.pyplot as plt


import pickle
import os

import numpy as np
from sklearn.decomposition import PCA

import sys
sys.path.append("..")
from src.image_data import ImageClassificationDataset

In [2]:
DATA_PATH = '/mnt/ssd/ronak/datasets/cifar10'
DEVICE = 'cuda:0'

In [4]:
def get_param_shapes(parameters):
    return [torch.tensor(p.shape) for p in parameters]

def get_num_parameters(param_shapes):
    return torch.tensor([torch.prod(s) for s in param_shapes]).sum().item()

In [8]:
model = convnext_base(weights=ConvNeXt_Base_Weights.IMAGENET1K_V1).to(DEVICE)
# train_nodes, eval_nodes = get_graph_node_names(model)

In [5]:
print(f"number of parameters in foundation model: {get_num_parameters(get_param_shapes(model.parameters()))}")

number of parameters in foundation model: 88591464


In [9]:
root = DATA_PATH

x_train = np.transpose(np.load(os.path.join(root, "x_train.npy")), axes=[0, 3, 1, 2])
y_train = np.load(os.path.join(root, "y_train.npy"))
print(x_train.shape)
print(x_train[0, :, :, 0])

batch_size = 1
transforms = ConvNeXt_Base_Weights.IMAGENET1K_V1.transforms()
train_dataset = ImageClassificationDataset(x_train, y_train, transforms)
dataloader = DataLoader(
    train_dataset, sampler=SequentialSampler(train_dataset), batch_size=batch_size
)

(50000, 3, 32, 32)
[[ 59  16  25  33  50  71  97 115 137 154 154 145 142 158 145 148 149 147
  152 145 143 143 141 143 149 172 202 216 220 208 180 177]
 [ 62  20  24  25  32  48  69  82 100 120 122 114 115 131 115 116 115 111
  114 105 104 104 102 103 107 128 157 174 182 170 139 144]
 [ 63  20  21  17  21  29  40  49  68  89  94  89  86  98  79  79  79  76
   80  72  66  64  65  72  74  76  82  87  91  96  96 116]]


In [10]:
all_image_features, all_idx = [], []
with torch.no_grad():
    for i, batch in tqdm(enumerate(dataloader)):
        idx, images, labels = batch
        output = model(images.to(DEVICE))
        break
        # all_image_features.append(image_features)
        all_idx.append(idx)
        
# all_image_features = torch.cat(all_image_features).cpu().detach().numpy()
# all_idx = torch.cat(all_idx).cpu().detach().numpy()

0it [00:00, ?it/s]/home/ronak/miniconda3/envs/dl/lib/python3.11/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
0it [00:00, ?it/s]


In [12]:
output.shape

torch.Size([1, 1000])